# Based on: ["Table Visualization"](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)

In [1]:
import pandas as pd
import numpy as np

The `DataFrame.style` property returns a `Styler` object.

`Styler` creates an HTML `<table>` and leverages CSS styling language to manipulate colors, fonts, borders, background, etc.

`Styler` objects have a `_repr_html_` method that automatically renders tables.

`Styler` can distinguish the **display value** from the **actual value**.

In [2]:
df = pd.DataFrame({
    "item": sorted(np.random.randint(1e4, 1e7, 5)),
    "price": np.random.rand(5) * 1e5,
    "description": ["<p>Some text</p>"] + [None] * 4
})
print(f"type: {type(df.style)}")
df.style

type: <class 'pandas.io.formats.style.Styler'>


,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


# 1. Formatting the Display

## 1.1 Formatting values
[Styler.format](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.format.html)

In [3]:
# Set the precision, missing number representation, thoudands separator
# and make HTML-safe
df.style.format(precision=2, na_rep="-", thousands="_", escape="html")

,item,price,description
0,962_636,76_081.14,<p>Some text</p>
1,4_499_800,21_932.97,-
2,7_768_738,32_555.43,-
3,9_645_611,13_425.63,-
4,9_679_232,17_202.75,-


In [4]:
df.style.format(formatter={
    "item": "{:.2g}",
    "price": "{:.4f}"
})

,item,price,description
0,9.6e+05,76081.1448,Some text
1,4.5e+06,21932.9678,None
2,7.8e+06,32555.4252,None
3,9.6e+06,13425.6285,None
4,9.7e+06,17202.7495,None


## 1.2 Hiding data

In [5]:
# Hide the index
df.style.hide_index()

item,price,description
962636,76081.144850,Some text
4499800,21932.967805,None
7768738,32555.425174,None
9645611,13425.628461,None
9679232,17202.749489,None


In [6]:
# Hide column headers
df.style.hide_columns()

0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


In [7]:
# Hide a subset of the rows/columns
df.style.hide_index([1, 3]).hide_columns(["description"])

,item,price
0,962636,76081.144850
2,7768738,32555.425174
4,9679232,17202.749489


# 2. Methods to Add Styles

## 2.1 [Styler.set_table_styles](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.set_table_styles.html)

Set the **table styles** included within the `<style>` HTML element.

Cannot be exported to Excel.

In [8]:
df.style.set_table_styles([
    {
        "selector": "th", 
        "props": "font-size: 1.4em"},
    {
        "selector": "tr:hover",
        "props": "background-color: cyan; font: oblique bold 1em serif;"
    }
])

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


## 2.2 [Styler.set_td_classes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.set_td_classes.html)

Set the `DataFrame` of strings(external CSS classes) added to the `class` attribute of **`<td>` HTML elements**.

Cannot be used on headers or exported to Excel.

In [9]:
classes_df = pd.DataFrame([
    ["min-val red LOOK-HERE", "", ""],      # classes for row0 values
    [""] * 3,                     # classes for row1 values
    [""] * 3,                     # classes for row2 values
    [""] * 3,                     # classes for row3 values
    ["max-val blue LOOK-HERE", None, ""]        # classes for row4 values

], index=df.index, columns=df.columns)
print(df.style.set_td_classes(classes_df).render())

<style type="text/css">
</style>
<table id="T_1dcf1_">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th class="col_heading level0 col0" >item</th>
      <th class="col_heading level0 col1" >price</th>
      <th class="col_heading level0 col2" >description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th id="T_1dcf1_level0_row0" class="row_heading level0 row0" >0</th>
      <td id="T_1dcf1_row0_col0" class="data row0 col0 min-val red LOOK-HERE" >962636</td>
      <td id="T_1dcf1_row0_col1" class="data row0 col1" >76081.144850</td>
      <td id="T_1dcf1_row0_col2" class="data row0 col2" ><p>Some text</p></td>
    </tr>
    <tr>
      <th id="T_1dcf1_level0_row1" class="row_heading level0 row1" >1</th>
      <td id="T_1dcf1_row1_col0" class="data row1 col0" >4499800</td>
      <td id="T_1dcf1_row1_col1" class="data row1 col1" >21932.967805</td>
      <td id="T_1dcf1_row1_col2" class="data row1 col2" >None</td>
    </tr>
    <tr>
      <th id="T_1dcf1_level0_r

## 2.3 [Styler.apply][1] & [Styler.applymap][2]

Add **direct internal CSS** to specific data cells.

Cannot be used on headers.

Can be exported to Excel.

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.apply.html
[2]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.applymap.html

In [10]:
def highlight_max(x, color):
    return np.where(x == np.nanmax(x.to_numpy()), f"color: {color};", None)

df.style.apply(highlight_max, color='orangered', subset=["price"])

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


In [11]:
def color_even(value, color):
    return f"color: {color};" if value % 2 == 0 else None

df.style.applymap(color_even, color="aqua", subset=["item"])

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


# 3. Table Styles

Recommended for **broad styling** e.g. entire rows or columns.

The styles are supplies as a `dict` or `list` of `dict`s; with CSS selectors as keys and CSS properties as values.

The CSS properties can be a single `str` or `list` of (property, value) `tuple`s.

As a convenience method (since version 1.2.0) you can pass a `dict` containing row/column keys to `Styler.set_table_styles`() 

In [12]:
headers = {
    "selector": "th",
    "props": "background-color: navy; color: white;"
}
cell_hover = {
    "selector": "td:hover",  # for row hover use "tr:hover" instead
    "props": [("background-color", "blue")]
}
df.style.set_table_styles([headers, cell_hover])

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


In [13]:
df.style.set_table_styles({
    "item": [
                {"selector": "td", "props": "color: blue;"}
            ],
    "description": [
                {"selector": "td", "props": "background-color: cyan;"}
            ]
})

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


# 4. Setting Classes and Linking to External CSS¶

## 4.1 Table Attributes

You can add a class to the main `<table>` using [Styler.set_table_attributes()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.set_table_attributes.html).

In [14]:
print(df.style.set_table_attributes('class="custom-table-cls"').render()[:100])

<style type="text/css">
</style>
<table id="T_36efe_" class="custom-table-cls">
  <thead>
    <tr>
 


## 4.2 Data cell CSS classes

[Styler.set_td_classes()][1] accepts a `DataFrame` with matching indices and columns to the underlying Styler’s DataFrame.

This styling `DataFrame` should contain strings as CSS-classes to add to individual data cells (or `None`).

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.set_td_classes.html

In [15]:
cell_classes = pd.DataFrame(
    [["item-cls", "price-cls", "description-cls"]] * 5,
    index=df.index,
    columns=df.columns
)

(
    # Simulate an external style sheet
    df.style.set_table_styles([
        {"selector": ".item-cls", "props": "color: purple;"},
        {"selector": ".price-cls", "props": "color: navy;"},
        {"selector": ".description-cls", "props": "color: #444;"},
    ])
    # Set table cell classes
    .set_td_classes(cell_classes)
)

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


# 5. Styler Functions

[Styler.applymap][1] (element-wise) and [Styler.apply][2] (column-/row-/table-wise) take a **function** and **keyword arguments** to render CSS style.

Useful for applying multiple, complex logic to data cells.

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.applymap.html
[2]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.apply.html

## 5.1 Styler.applymap - element-wise

In [16]:
df2 = pd.DataFrame(np.random.standard_normal((8, 4)), columns=list("ABCD"))


def style_negatives(value, props=""):
    return props if value < 0 else None


df2.style.applymap(style_negatives, props="background-color: cyan; color: red;")

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


## 5.2 `Styler.apply` - column-wise(axis=0)

In [17]:
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')


df2.style.apply(highlight_max, props='font-size: 1.5em;', axis=0)

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


## 5.3 `Styler.apply` -  row-wise(axis=1)

In [18]:
df2.style.apply(highlight_max, props="background-color: yellow;", axis=1)

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


## 5.4 `Styler.apply` - table-wise(axis=None)

In [19]:
df2.style.apply(highlight_max, props="color: lime; font-size: 1.6em;", axis=None)  

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


# 6. Tooltips and Captions

[Styler.set_caption][1] sets the text added to a `<caption>` HTML element.

[Styler.set_tooltips][2] sets cell tootltips from a string based `DataFrame`.

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.set_caption.html
[2]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.set_tooltips.html

In [20]:
df.style.set_caption("Table of Product Information")

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


In [21]:
tooltips = pd.DataFrame(
    [
        ["item id", "price in Ksh.", "Some text"],
        ["item id", "price in Ksh.", "Nothing to see here"],
        ["item id", "price in Ksh.", "Nothing to see here"],
        ["item id", "price in Ksh.", "Nothing to see here"],
        ["item id", "price in Ksh.", "Nothing to see here"]
    ],
    columns=df.columns,
    index=df.index
)

df.style.set_tooltips(tooltips)

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


# 7. Slicing

Achieved using the `subset` keyword argument, which specifies rows/columns.

To select a subset both column-wise and row-wise, use `pd.IndexSlice`.

In [22]:
df2.style.applymap(style_negatives, props="color: red;", subset=['B', 'C'])

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


In [23]:
df2.style.apply(highlight_max, props="background-color: cyan;",
                subset=pd.IndexSlice[0:4, ['A', 'D']], axis=1)

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


# 8. Optimization

Only useful if:

* rendering and styling a **very large HTML table**
* **dynamically creating** part of online user interfaces (boost **network performance**)

## 8.1 Remove UUID and cell_ids

In [24]:
from pandas.io.formats.style import Styler

# Makes id's shorter, and remove id's from <td> elements
print(Styler(df, uuid_len=0, cell_ids=False).render())

<style type="text/css">
</style>
<table id="T__">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th class="col_heading level0 col0" >item</th>
      <th class="col_heading level0 col1" >price</th>
      <th class="col_heading level0 col2" >description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th id="T__level0_row0" class="row_heading level0 row0" >0</th>
      <td  class="data row0 col0" >962636</td>
      <td  class="data row0 col1" >76081.144850</td>
      <td  class="data row0 col2" ><p>Some text</p></td>
    </tr>
    <tr>
      <th id="T__level0_row1" class="row_heading level0 row1" >1</th>
      <td  class="data row1 col0" >4499800</td>
      <td  class="data row1 col1" >21932.967805</td>
      <td  class="data row1 col2" >None</td>
    </tr>
    <tr>
      <th id="T__level0_row2" class="row_heading level0 row2" >2</th>
      <td  class="data row2 col0" >7768738</td>
      <td  class="data row2 col1" >32555.425174</td>
      <td  class="data row2 

## 8.2. Use table styles

Prefer `Styler.set_table_styles` to other styling functions. Wider coverage == more efficiency.

## 8.3. Set classes instead of using Styler functions

For large `DataFrames` where the same style is applied to many cells it can be more efficient to declare the styles as classes, and then apply those classes to data cells.

## 8.4. Don’t use tooltips

Tooltips require cell_ids to work, and they generate extra HTML elements for every data cell.

## 8.5. If every byte counts use string replacement

You can remove unnecessary HTML, or shorten the default class names with string replace functions.

# 9. Builtin Styles

[highlight_null][1]

[highlight_min][2]

[highlight_max][3]

[highlight_between][4]

[highlight_quantile][5]

[background_gradient][6]

[text_gradient][7]

[bar][8]

[set_properties][9]

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.highlight_null.html
[2]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.highlight_min.html
[3]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.highlight_max.html
[4]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.highlight_between.html
[5]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.highlight_quantile.html
[6]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.background_gradient.html
[7]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.text_gradient.html
[8]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.bar.html
[9]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.set_properties.html

In [25]:
df.style.highlight_null(null_color="#444")

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


In [26]:
(
    df2.style
    .highlight_min(color="cyan", axis=1)
    .highlight_max(props="font: oblique normal 1.4em serif;", axis=1)
)

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


In [27]:
# Highlight values in the range [-0.5, 0.5]
df2.style.highlight_between(left=-0.5, right=0.5)

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


In [28]:
# Highlight values within 5th and 95 percentiles in each column
df2.style.highlight_quantile(q_left=0.05, q_right=0.95, color="teal")

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


In [29]:
df2.style.background_gradient(cmap="Greens")

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


In [30]:
df2.style.text_gradient()

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


In [31]:
df2.style.bar(align="mid", color=["aquamarine", "lawngreen"])

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


In [32]:
df.style.set_properties(**{
    "background-color": "#ccc",
    "font-family": "Courier, mono"
})

,item,price,description
0,962636,76081.144850,Some text
1,4499800,21932.967805,None
2,7768738,32555.425174,None
3,9645611,13425.628461,None
4,9679232,17202.749489,None


# 10. Sharing Styles

You can access an existing style using the `export` method, and apply it to a different dataset with the `use` method.

In [33]:
df_style1 = df.style.set_properties(**{
    "background-color": "#ccc",
    "font-family": "Courier, mono"
})
df2.style.use(df_style1.export())

,A,B,C,D
0,2.331257,-0.194038,0.510170,0.280956
1,1.786892,1.313914,-2.623781,1.050852
2,-0.372940,-0.818885,-2.518880,-2.763697
3,-0.270809,1.037445,0.452839,-0.623145
4,-1.537236,-0.298211,-0.329748,0.618655
5,0.695964,-1.219013,0.264508,-1.843860
6,0.499248,0.522890,-0.936298,-0.872510
7,0.778595,0.943862,-0.049274,0.855342


# 11. Limitations¶

* DataFrame only (use Series.to_frame().style)
* The index and columns must be unique
* No large repr, and construction performance isn’t great
* You can only style the values, not the index or columns (except with set_table_styles)
* You can only apply styles, you can’t insert new HTML entities

# 12. Sticky Headers

In [34]:
wide_df = pd.DataFrame(np.random.randn(5, 20))

# Make index visible even while you scroll
wide_df.style.set_sticky(axis="index")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-1.197518,0.225025,-1.099760,-0.080962,0.458308,-1.188131,1.054379,-0.302538,0.178420,-0.468877,2.824498,0.782276,0.162337,-0.498562,0.236985,1.270816,0.647515,1.077497,-0.086785,0.455556
1,-0.818072,-0.641942,-0.136456,-1.084640,-0.393117,-0.748519,-0.276304,-1.707759,-0.713202,-0.489235,-0.195746,-0.328911,0.402879,-0.682011,-0.422344,-0.730981,0.506730,1.047125,-0.909036,0.506205
2,-0.980651,-0.110627,0.597167,0.892743,0.720027,-1.728988,-0.362544,-0.700567,0.163256,0.308097,0.001140,-0.326917,2.513283,-0.179625,1.347440,1.332675,-2.202856,-0.370354,-1.657989,0.673420
3,-1.950617,-0.859735,-0.999326,2.038534,1.645869,0.039517,0.893883,1.001335,-0.579768,0.515796,-0.537175,1.939652,0.322794,-1.257155,1.123147,1.614573,0.529431,-1.436479,-0.465029,1.092206
4,-0.055548,-1.659597,0.054728,-0.817872,0.047369,0.470350,-0.344604,-1.196484,-1.090695,0.873500,-1.508618,-0.656213,0.747765,0.892607,-0.543063,-0.685157,0.190507,0.679171,1.284744,-0.751022


In [35]:
wide_df.T.style.set_sticky(axis="columns")

,0,1,2,3,4
0,-1.197518,-0.818072,-0.980651,-1.950617,-0.055548
1,0.225025,-0.641942,-0.110627,-0.859735,-1.659597
2,-1.099760,-0.136456,0.597167,-0.999326,0.054728
3,-0.080962,-1.084640,0.892743,2.038534,-0.817872
4,0.458308,-0.393117,0.720027,1.645869,0.047369
5,-1.188131,-0.748519,-1.728988,0.039517,0.470350
6,1.054379,-0.276304,-0.362544,0.893883,-0.344604
7,-0.302538,-1.707759,-0.700567,1.001335,-1.196484
8,0.178420,-0.713202,0.163256,-0.579768,-1.090695
9,-0.468877,-0.489235,0.308097,0.515796,0.873500


# 13. Export to Excel

[Styler.to_excel](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.to_excel.html)

Only works with `Styler.apply` and/or `Styler.applymap` methods. CSS properties supported:

* background-color
* color
* font-family
* font-style
* font-weight
* text-align
* text-decoration
* vertical-align
* white-space: nowrap

Only CSS-2 named colors and hex colors of the form `#rgb` or `#rrggbb` are currently supported.

Pseudo-CSS properties for excel specific style:

* number-format

In [36]:
(
    df2.style.applymap(style_negatives, props="color: red;")
    .to_excel('../data/styled.xlsx', engine='openpyxl')
)

# 14. Export to LaTeX

[Styler.to_latex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.to_latex.html)

In [37]:
df.style.to_latex()

'\\begin{tabular}{lrrl}\n{} & {item} & {price} & {description} \\\\\n0 & 962636 & 76081.144850 & <p>Some text</p> \\\\\n1 & 4499800 & 21932.967805 & None \\\\\n2 & 7768738 & 32555.425174 & None \\\\\n3 & 9645611 & 13425.628461 & None \\\\\n4 & 9679232 & 17202.749489 & None \\\\\n\\end{tabular}\n'